## 상태관리
* State는 LangGraph로 구현된 시스템이 처리하는 모든 정보를 담은 객체
* 노드 간에 데이터를 전달하고 축적하는 역할

**상태가 필요한 이유**
1. **노드는 독립적으로 처리되며 서로 직접 통신하지 않음**
2. **상태를 이용하여 간접 통신함**
3. **병렬 실행 가능한 노드들은 같은 super-step에서 동시에 실행됨**


In [23]:
# Sample State
from typing import TypedDict
class ChatState(TypedDict):
    user_message: str # 현재 사용자의 메시지
    chat_history: list # 이전대화 기록
    user_info: dict # 사용자 정보
    system_status: str # 시스템 상태

### **상태관리의 중요성**
1. 일관성 유지
    1. 복잡한 워크플로우에서 맥락을 일관데게 유지하여 UX향상
        * 멀티턴 대화를 구현하는 요소, 정보가 손실되지 않고 유지되는것
2. 복잡한 작업 처리
    1. 다단계 작업이나 다주제 대화를 원활하게 관리할 수 있음
        * 여러 단계로 구성된 작업에서 각 단계의 결과가 다음 단계의 입력값이 됨
        * 상태 관리가 이를 구현하는 데 가능하게 함
3. 오류 복구 및 중단점 관리
    1. 시스템 장애 시 복구가 용이하며, 특정 지점에서 실행을 중단 및 재개할 수 있음
        * `Checkpointing`을 이용해서 구현할 수 있으며, 문제 발생시 마지막 체크포인트에서 재개
4. 성능 최적화
    1. 불필요한 계산을 줄여 시스템 전반의 성능 향상
        * 상태에 계산 결과를 캐싱하면 반복 연산 하지 않음

In [24]:
# 대화 맥락 유지 예시
def generate_contextual_response(
    user_message:str,
    context:str,
    ):
    pass


def conversation_node(state: ChatState):
    # 이전 대화 내용을 참조하여 일관된 응답 생성
    context = "\n".join(state["chat_history"][-3:])  # 최근 3개 대화
    response = generate_contextual_response(state["user_message"], context)
    return {"ai_response": response}


### 상태 스키마 설계
**설계 원칙**
1. 명확성
    * 각 필드의 목적과 사용법이 명확해야함
    * 명확성은 코드의 가독성과 유지보수성을 높이고, 타입힌팅과 문서화를 통해 사용법 명시해야함
2. 캡슐화
    * 내부 처리용 데이터와 외부 인터페이스 구분
    * 객체지향형 프로그래밍 의 핵심 원칙 중 하나
    * 입/출력 스키마를 분리하여 내부 구현을 숨기고 명확한 인터페이스 제공 가능
3. 단일 책임 원칙
    * 각 상태 스키마는 하나의 명확한 목적을 가져야함
    * 하나의 스키마가 많은 책임을 갖게 되면 복잡도 증가, 유지보수 어려워짐

### 상태 스키마 유형
#### 기본 스키마
**개요**
* 입/출력이 동일한 단일 스키마를 사용하는 형태
* 가장 단순하고 직관적이며, 데이터 프름이 명확하고 디버깅 쉬움
* 사스템 복잡도 증가시 불필요한 필드 많아질 수 있음
* 단순한 선형적 워크플로우, 프로토 타입, 간단한 챗봇 (Q&A)에서 사용 가능

**장단점**
| 장점|단점|
|----|-----|
| 구현이 간단하고 이해하기 쉬움 |복잡한 시스템에서는 유연성 부족|
|디버깅이 용이함|모든 노드가 전체 상태에 접근 가능 (보안 문제 가능)|
|빠른 개발 가능|스키마가 비대해질 수 있음|

In [ ]:
from typing import TypedDict
from typing import Annotated
from operator import add

class BasicState(TypedDict):
    user_input: str                                    # 사용자 입력
    ai_response: str                                   # AI 응답
    conversation_history: Annotated[list[str], add]    # 대화 이력 (누적)

# 실사용 예시
def chatbot_node(state: BasicState) -> dict:
    response = f"사용자님이 '{state['user_input']}'라고 하셨군요!"
    return {
            "ai_response": response,
            "conversation_history": [
                f"User: {state['user_input']}", f"AI: {response}"
            ]
    }

#### 명시적 입출력 스키마
**개요**
* 입/출력을 별도로 정의하여 인터페이스를 명확하게 제어함
* API설계에서 자주 사용됨
* 외부에서 보는 인터페이스와 내부 처리용 데이터를 분리하여</br>
내부 구현의 변경이 어려운 외부 인터페이스에 영향을 주지 않음

**용례**
* API 같은 명확한 인터페이스가 필요한 경우
* 외부 시스템과 연동할 때
* 내부 처리 과정을 숨기고 싶을 때
* 버전 관리가 중요한 시스템

**장단점**
|장점|단점|
|---|---|
|명확한 API 인터페이스|설정이 다소 복잡함|
|내부 구현 캡슐화|더 많은 코드 필요|
|버전 관리 용이|상태 변환 로직 필요|
|보안 향상 (내부 데이터 노출 방지)||

In [ ]:
# Input State
class InputState(TypedDict):
    question:str

# Output State
class OutputState(TypedDict):
    answer:str
    
class OverallState(InputState, OutputState):
    intermediate_data:str
    search_reseult:list[str]
    confidence_socre: float

# 사용 예시: 검색 기반 QA 시스템
def search_node(state: InputState) -> dict:
    results = ["결과1", "결과2", "결과3"]
    return {
        "search_results": results,
        "intermediate_data": f"'{state['question']}'에 대한 검색 완료"
    }

def answer_node(state: OverallState) -> OutputState:
    answer = f"검색 결과를 바탕으로: {state['search_results'][0]}"
    return {"answer": answer}  

#### 다중 스키마
**개요**
* 내부 노드 간 통신을 위한 private 스키마를 포함하는 복잡한 구조
* 대규모 시스템은 각 처리 단계마다 다른 데이터 구조가 필요할 수 있음
* 각 단계의 최적화된 스키마를 사용하면서 데이터 흐름을 체계적으로 관리할 수 있음

**용례**
* RAG(Retrieval-Augmented Generation) 파이프라인
* 다단계 처리가 필요한 복잡한 시스템
* 각 단계별로 다른 데이터 구조가 필요한 경우
* 마이크로서비스 아키텍처

**장단점**
|장점|단점|
|---|---|
|각 단계별 최적화된 데이터 구조|구조가 복잡함|
|높은 모듈화|설계 시 신중함 필요|
|재사용성 증가|상태 변환 오버헤드|
|단계별 독립적 테스트 가능|디버깅이 어려울 수 있음|

#### 스키마 선택 가이드
| 상황 | 추천 스키마 | 이유 |
| :--- | :--- | :--- |
| 간단한 챗봇 | 기본 스키마 | 구현이 단순하고 빠른 프로토타이핑이 가능함|
| 마이크로서비스 API | 명시적 입출력 | 인터페이스가 명확하고 버전 관리가 용이함|
| RAG 시스템 | 다중 스키마 | 각 단계별로 최적화가 가능하며 복잡한 데이터 흐름을 처리할 수 있음|
| 멀티에이전트 시스템 | 다중 스키마 | 에이전트별로 독립적인 상태 관리가 가능함|
| 실시간 스트리밍 | 기본 스키마 | 지연 시간을 최소화할 수 있음|

In [47]:
# 전체 상태 (공개 인터페이스)
class OverallState(TypedDict):
    question: str                    # 사용자 질문
    answer: str                      # 최종 답변

# 쿼리 생성 단계의 출력
class QueryOutputState(TypedDict):
    query: str                       # 생성된 검색 쿼리
    query_type: str                  # 쿼리 유형

# 문서 검색 단계의 출력
class DocumentOutputState(TypedDict):
    docs: list[str]                  # 검색된 문서들
    relevance_scores: list[float]    # 관련성 점수들

# 답변 생성 단계의 입력 (여러 스키마 조합)
class GenerateInputState(OverallState, DocumentOutputState):
    pass


## Reducer
* 함수형 프로그래밍에서 정의된 개념
* 상태 업데이트 로직을 정의하는 핵심 메커니즘
* JS의 Redux, React의 useReducer와 유사한 개념

### 필요한 이유
1. 여러 노드가 동시에 또는 순차적으로 실행될 때, 각 노드의 출력을 상태에 통합하는 방법이 필요함
2. 단순히 덮어쓰기만 하면 이전 정보가 손실될 수 있습니다
3. 리듀서를 통해 누적, 병합, 최대값 유지 등 다양한 업데이트 전략을 구현할 수 있습니다

### 동작 원리
* 노드는 전체 상태가 아닌 업데이트 할 부분만 반환
* 업데이트된 부분을 기존 상태와 결합

### 종류
1. 기본 리듀서(덮어쓰기)
    * 새로운 값이 입력되면 기존 값은 완전히 사라짐
    * 가작 단순하고 직관적인 방식

2. 내장 리듀서
    * `operator.add():[str,int,float]`
      * python의 내장함수로 이전 상태값과 업데이트된 상태값을 추가하는 리듀서
    * `add_messages: Annotated[list,add_messages]`
      * LangChain에 포함된 리듀서, 메시지 객체를 위한 리듀서
3. 사용자 정의 리듀서
    * 특정 비즈니스 로직을 구현할 수 있는 리듀서

### 선택 가이드
1. 덮어쓰기 (기본): 상태, 설정값, 현재 사용자 등
2. operator.add: 로그, 메시지 히스토리, 이벤트 목록
3. max/min: 최고/최저 점수, 임계값 관리
4. 사용자 정의: 복잡한 비즈니스 로직, ID 기반 업데이트

### 주의사항 
* 노드는 전체 상태가 아닌 변경하려는 부분만 반환
* 리듀서는 각 키별로 독립적으로 동작
* 병렬 노드 실행 시 같은 키를 업데이트하면 충돌 가능 (적절한 리듀서로 해결)
* `Annotated[타입, 리듀서]` 형식으로 리듀서 지정

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from operator import add
from langgraph.graph import StateGraph, START, END
from langgraph.graph import MessagesState

class State(MessagesState):
    message_count: int                           # 기본 리듀서 (덮어쓰기)
    conversation: Annotated[list[str], add]      # add 리듀서 (리스트 연결)

def node_1(state: State) -> dict:
    """
    첫 번째 노드: 대화 시작
    - message_count를 1로 설정
    - conversation에 인사말 추가
    """
    print(f"Node 1 - 현재 대화: {state.get('conversation', [])}")
    return {
        "message_count": 1,
        "conversation": ["안녕하세요!"]
    }

def node_2(state: State) -> dict:
    """
    두 번째 노드: 대화 이어가기
    - message_count는 업데이트하지 않음 (이전 값 유지)
    - conversation에 새 메시지 추가 (add 리듀서로 누적)
    """
    print(f"Node 2 - 현재 대화: {state['conversation']}")
    # message_count를 반환하지 않으므로 node_1에서 설정한 값이 유지됨
    return {
        "conversation": ["어떻게 도와드릴까요?"]
    }

# 그래프 구성
graph = StateGraph(State)
graph.add_node("node_1", node_1)
graph.add_node("node_2", node_2)
graph.add_edge(START, "node_1")  # 시작 -> node_1
graph.add_edge("node_1", "node_2")  # node_1 -> node_2
graph.add_edge("node_2", END)  # node_2 -> 종료

compiled_graph = graph.compile()

# 실행
initial_state = State(
    {
        "message_count": 0,
        "conversation": [],
        "messages": []
    }
)
result = compiled_graph.invoke(initial_state)

print("\n=== 최종 결과 ===")
print(f"메시지 수: {result['message_count']}")
print(f"대화 내용: {result['conversation']}")

# 출력:
# Node 1 - 현재 대화: []
# Node 2 - 현재 대화: ['안녕하세요!']
#
# === 최종 결과 ===
# 메시지 수: 1
# 대화 내용: ['안녕하세요!', '어떻게 도와드릴까요?']


Node 1 - 현재 대화: []
Node 2 - 현재 대화: ['안녕하세요!']

=== 최종 결과 ===
메시지 수: 1
대화 내용: ['안녕하세요!', '어떻게 도와드릴까요?']
